<a href="https://colab.research.google.com/github/Vakhranev/Compling/blob/master/Diploma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pynput

     |████████████████████████████████| 102kB 4.4MB/s 
     |████████████████████████████████| 184kB 8.0MB/s 
  Created wheel for evdev: filename=evdev-1.4.0-cp37-cp37m-linux_x86_64.whl size=97417 sha256=11d79117b0742dec3dc63a9a5dba9b2b36e5a0a0c2c3d1338a46ca9536b751d8
  Stored in directory: /root/.cache/pip/wheels/66/35/1d/38b2de1421ede48ebf0107faea56163d07059110b116a794a3
Successfully built evdev


In [ ]:
!pip install patool

     |████████████████████████████████| 81kB 4.4MB/s 


In [ ]:
import patoolib

In [ ]:
archive = '/content/drive/My Drive/lenta-ru-news.rar'

In [ ]:
patoolib.extract_archive(archive)

patool: Extracting /content/drive/My Drive/lenta-ru-news.rar ...
patool: running /usr/bin/unrar x -- "/content/drive/My Drive/lenta-ru-news.rar"
patool:     with cwd='./Unpack_ji7t4s0d'
patool: ... /content/drive/My Drive/lenta-ru-news.rar extracted to `lenta-ru-news.csv'.


'lenta-ru-news.csv'

In [ ]:
corpora = pd.read_csv('lenta-ru-news.csv',engine='python', error_bad_lines=False)

In [ ]:
corpora.head()

,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая,1914/09/17
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая,1914/09/18


In [ ]:
corpora['text'].head()

0    Бои у Сопоцкина и Друскеник закончились отступ...
1    Министерство народного просвещения, в виду про...
2    Штабс-капитан П. Н. Нестеров на днях, увидев в...
3    Фотограф-корреспондент Daily Mirror рассказыва...
4    Лица, приехавшие в Варшаву из Люблина, передаю...
Name: text, dtype: object

In [12]:
import re

In [ ]:
new_corpora = []
for text in corpora['text']:
  split_regex = re.compile(r'[.|!|?|…]')
  sentences = filter(lambda t: t, [t.strip() for t in split_regex.split(str(text))])
  for sentence in sentences:
    new_corpora.append(sentence)
print(len(new_corpora))

9233828


In [ ]:
new_corpora[0:10]

['Бои у Сопоцкина и Друскеник закончились отступлением германцев',
 'Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью',
 'В артиллерийском бою принимают участие тяжелые калибры',
 'С раннего утра 14 сентября огонь достиг значительного напряжения',
 'Попытка германской пехоты пробиться ближе к крепости отражена',
 'В Галиции мы заняли Дембицу',
 'Большая колонна, отступавшая по шоссе от Перемышля к Саноку, обстреливалась с высот нашей батареей и бежала, бросив парки, обоз и автомобили',
 'Вылазки гарнизона Перемышля остаются безуспешными',
 'При продолжающемся отступлении австрийцев обнаруживается полное перемешивание их частей, захватываются новые партии пленных, орудия и прочая материальная часть',
 'На перевале Ужок мы разбили неприятельский отряд, взяли его артиллерию и много пленных и, продолжая преследовать, вступили в пределы Венгрии']

In [ ]:
!pip install ufal.udpipe

     |████████████████████████████████| 307kB 5.4MB/s 
  Created wheel for ufal.udpipe: filename=ufal.udpipe-1.2.0.3-cp37-cp37m-linux_x86_64.whl size=5626592 sha256=a0300cbf720059dc2d357626a385da73c9b2abf4f4db86332c9707fa55989443
  Stored in directory: /root/.cache/pip/wheels/0c/9d/db/6d3404c33da5b7adb6c6972853efb6a27649d3ba15f7e9bebb
Successfully built ufal.udpipe


In [ ]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=42cd3ccdf51b5f9f75ad3f307f35f08ad26da582a06a120329a7bb35e46fcdf5
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [ ]:
import wget
import sys
udpipe_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
modelfile = wget.download(udpipe_url)

In [ ]:
def process(pipeline, text='Строка', keep_pos=True, keep_punct=False):
   entities = {'PROPN'}
   named = False  # переменная для запоминания того, что нам встретилось имя собственное
   memory = []
   mem_case = None
   mem_number = None
   tagged_propn = []
# обрабатываем текст, получаем результат в формате conllu:
   processed = pipeline.process(text)
# пропускаем строки со служебной информацией:
   content = [l for l in processed.split('\n') if not l.startswith('#')]
# извлекаем из обработанного текста леммы, тэги и морфологические характеристики
   tagged = [w.split('\t') for w in content if w]
   for t in tagged:
       if len(t) != 10: # если список короткий — строчка не содержит разбора, пропускаем
           continue
       (word_id,token,lemma,pos,xpos,feats,head,deprel,deps,misc) = t 
       if not lemma or not token: # если слово пустое — пропускаем
           continue
       if pos in entities: # здесь отдельно обрабатываем имена собственные — они требуют особого обращения
           if '|' not in feats:
               tagged_propn.append('%s_%s_%s_%s_%s_%s' % (word_id, token, lemma, pos, head, deprel))
               continue
           morph = {el.split('=')[0]: el.split('=')[1] for el in feats.split('|')}
           if 'Case' not in morph or 'Number' not in morph:
               tagged_propn.append('%s_%s_%s_%s_%s_%s' % (word_id, token, lemma, pos, head, deprel))
               continue
           if not named:
               named = True
               mem_case = morph['Case']
               mem_number = morph['Number']
           if morph['Case'] == mem_case and morph['Number'] == mem_number:
               memory.append(lemma)
               if 'SpacesAfter=\\n' in misc or 'SpacesAfter=\s\\n' in misc:
                   named = False
                   past_lemma = '::'.join(memory)
                   memory = []
                   tagged_propn.append(past_lemma + '_PROPN ')
           else:
               named = False
               past_lemma = '::'.join(memory)
               memory = []
               tagged_propn.append(past_lemma + '_PROPN ')
               tagged_propn.append('%s_%s_%s_%s_%s_%s' % (word_id, token, lemma, pos, head, deprel))
       else:
           if not named:
               tagged_propn.append('%s_%s_%s_%s_%s_%s' % (word_id, token, lemma, pos, head, deprel))
           else:
               named = False
               past_lemma = '::'.join(memory)
               memory = []
               tagged_propn.append(past_lemma + '_PROPN ')
               tagged_propn.append('%s_%s_%s_%s_%s_%s' % (word_id, token, lemma, pos, head, deprel))
   if not keep_punct: # обрабатываем случай, когда пользователь попросил не сохранять пунктуацию (по умолчанию она сохраняется)
       tagged_propn = [word for word in tagged_propn if word.split('_')[1] != 'PUNCT']
   if not keep_pos:
       tagged_propn = [word.split('_')[0] for word in tagged_propn]
   return tagged_propn

In [ ]:
from ufal.udpipe import Model, Pipeline
import os
import re
def tag_ud(text='Текст нужно передать функции в виде строки!', modelfile='udpipe_syntagrus.model'):
   udpipe_model_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
   udpipe_filename = udpipe_model_url.split('/')[-1]
   if not os.path.isfile(modelfile):
       print('UDPipe model not found. Downloading...', file=sys.stderr)
       wget.download(udpipe_model_url)
   #print('\nLoading the model...', file=sys.stderr)
   model = Model.load(modelfile)
   process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')
   #print('Processing input...', file=sys.stderr)
   lines = text.split('\n')
   tagged = []
   for line in lines:
# line = unify_sym(line.strip()) # здесь могла бы быть ваша функция очистки текста
       output = process(process_pipeline, text=line)
       tagged_line = ' '.join(output)
       tagged.append(tagged_line)
   return '\n'.join(tagged)
text = new_corpora[38667:40000]
for sentence in text:
   processed_text = tag_ud(text=sentence, modelfile=modelfile)
   #print(processed_text[:350])
   with open('my_text.txt', 'a', encoding='utf-8') as out:
       out.write(processed_text + '\n')

KeyboardInterrupt: ignored

In [1]:
import nltk

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
 new_corpora = open("my_text (9).txt", encoding="UTF-8", errors='ignore')

In [4]:
tokens=[]
for sentence in new_corpora:
  tokens.append(nltk.word_tokenize(sentence))

In [5]:
tokens[0:10]

[['1_Бои_бой_NOUN_6_nsubj',
  '2_у_в_ADP_3_case',
  'Сопоцкин_PROPN',
  '4_и_и_CCONJ_5_cc',
  'Друскеник_PROPN',
  '6_закончились_заканчиваться_VERB_0_root',
  '7_отступлением_отступление_NOUN_6_obl',
  '8_германцев_германец_NOUN_7_nmod'],
 ['1_Неприятель_неприятель_NOUN_8_nsubj',
  '2_',
  ',',
  '_',
  ',',
  '_PUNCT_1_punct',
  '3_приблизившись_приближаться_VERB_8_advcl',
  '4_с_с_ADP_5_case',
  '5_севера_север_NOUN_3_obl',
  '6_к_к_ADP_7_case',
  'Осовц_PROPN',
  '8_начал_начинать_VERB_0_root',
  '9_артиллерийскую_артиллерийский_ADJ_10_amod',
  '10_борьбу_борьба_NOUN_8_obj',
  '11_с_с_ADP_12_case',
  '12_крепостью_крепость_NOUN_10_nmod'],
 ['1_В_в_ADP_3_case',
  '2_артиллерийском_артиллерийский_ADJ_3_amod',
  '3_бою_бой_NOUN_4_obl',
  '4_принимают_принимать_VERB_0_root',
  '5_участие_участие_NOUN_4_obj',
  '6_тяжелые_тяжелый_ADJ_7_amod',
  '7_калибры_калибр_NOUN_5_obj'],
 ['1_С_с_ADP_3_case',
  '2_раннего_ранний_ADJ_3_amod',
  '3_утра_утро_NOUN_7_obl',
  '4_14_14_NUM_7_nummod',
  '

In [6]:
law_corpora=[]
for sentence in tokens:
  for token in sentence:
    if "_судья_" in token:
      if sentence not in law_corpora:
        law_corpora.append(sentence)
    elif "_адвокат_" in token:
      if sentence not in law_corpora:
        law_corpora.append(sentence)
    elif "_прокурор_" in token:
      if sentence not in law_corpora:
        law_corpora.append(sentence)
    else:
      pass
print(len(law_corpora))

2602


In [62]:
law_corpora[0:10]

[['1_Швейцарский_швейцарский_ADJ_3_amod',
  '2_федеральный_федеральный_ADJ_3_amod',
  '3_прокурор_прокурор_NOUN_7_nsubj',
  '5_дель_дель_PART_7_advmod',
  '6_Понти_Понти_NUM_5_nummod',
  '7_заявила_заявлять_VERB_0_root',
  '9_что_что_SCONJ_11_mark',
  '11_работал_работать_VERB_7_ccomp',
  '12_на_на_ADP_13_case',
  '13_шефа_шеф_NOUN_11_obl',
  '14_разведывательной_разведывательный_ADJ_15_amod',
  '15_службы_служба_NOUN_13_nmod',
  '18_и_и_CCONJ_19_cc',
  '19_получил_получать_VERB_11_conj',
  '20_5,8_5,8_NUM_21_nummod',
  '21_миллионов_миллион_NOUN_19_obl',
  '22_долларов_доллар_NOUN_21_nmod',
  '23_от_от_ADP_24_case',
  '24_министерства_министерство_NOUN_19_obl',
  '25_обороны_оборона_NOUN_24_nmod',
  '26_для_для_ADP_27_case',
  '27_покупки_покупка_NOUN_24_nmod',
  '28_огромного_огромный_ADJ_29_amod',
  '29_количества_количество_NOUN_27_nmod',
  '30_оружия_оружие_NOUN_29_nmod'],
 ['1_Адвокат_адвокат_NOUN_5_nsubj',
  '4_Седу_Сед_PROPN_3_flat',
  'name',
  '5_заявил_заявлять_VERB_0_root',

In [8]:
with open("law_corpora.txt", "w") as file:
  print(law_corpora, file=file, sep="\n")
file.close()

In [9]:
count=0
for sentence in tokens:
  count += len(sentence)
print(count)

5860022


In [10]:
count=0
for sentence in law_corpora:
  count += len(sentence)
print(count)

93703


In [11]:
len(tokens)

201486

In [101]:
for sentence in law_corpora:
  for i in sentence:
    listik=[]
    listik=i.split('_')
    #print(listik)
    if len(listik) < 3:
      sentence.remove(i)

In [110]:
diff={}
for sentence in law_corpora:
  for i in sentence:
    w=[]
    e=[]
    r=[]
    if "_судья_" in i:
      w=i.split(sep="_")
      index=w[0]
      if index == None:
        pass
      else:
        for m in sentence:
          e=m.split(sep="_")
          if len(w)<3 or len(e)<3:
            pass
          else:
            if e[2] == w[2]:
              pass
            else:
              small_diff=[]
              index2=e[0]
              if index2 == None:
                pass
              else:
                d = abs(int(index2)-int(index))-1
                for sentence in law_corpora:
                  if i in sentence:
                    #print(sentence)
                    for q in sentence:
                      #print(q)
                      r=q.split(sep="_")
                      if len(r)<3 or len(e)<3:
                        pass
                      else:
                        if r[2] == e[2]:
                          index=w[0]
                          if index == None:
                            pass
                          else:
                            index2=r[0]
                            if index == None:
                              pass
                            else:
                              d = abs(int(index2)-int(index))-1
                              small_diff.append(d)
                        else:
                          pass
                    if len(r) >= 3:
                      diff[r[2]]=small_diff
print(diff)

{'слово': [7], 'аноним': [16], 'собственность': [16], 'компания': [21], 'дело': [21], 'reuters': [27], 'посредник': [25], 'farm': [25], 'стража': [21], 'от': [25], 'в': [21], 'преподавание': [12, 21], 'Telegraph': [12, 21], 'бить': [12, 21], 'стерлинг': [12, 21], 'год': [20], 'истина': [12, 21], 'допускать': [12, 21], 'провинция': [12, 21], 'экстрадиция': [12, 21], 'тюрьма': [21], 'закон': [8], 'судья': [4], 'выносить': [0, 0, 0], 'март': [0, 0, 0], 'вопрос': [38, 32], 'против': [0], 'Ю': [], 'банкротство': [31], 'проступок': [16], 'кладбище': [16], 'содержание': [13], 'диск': [13], 'свобода': [13], 'адвокат': [13], 'случай': [22, 17, 26, 9, 15], 'банк': [22, 17, 26, 9, 15], 'полиция': [8], 'премьер-министр': [39], 'потребитель': [39], 'и': [21], 'хвалить': [39], 'надзор': [39], 'Бэрри': [39], 'сеть': [20], 'требование': [20], 'пенальти': [20], 'суверенитет': [20], 'сам': [20], 'команда': [20], 'полномочие': [20], 'телекомпания': [20], 'власть': [20], 'спутник': [20], 'больница': [20],

In [111]:
len(diff)

376